In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
import pytorch3d.io
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex

import trimesh

from src.util import make_faces

In [66]:
n = 2
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)
mesh

In [28]:
mesh.verts_packed().shape, mesh.faces_packed().shape

(torch.Size([24, 3]), torch.Size([12, 3]))

In [29]:
m = trimesh.Trimesh(vertices=vert, faces=faces)
m.export('./data/cube.stl');

In [30]:
sides['front'].shape

torch.Size([2, 2, 3])

In [31]:
make_faces(n, n).shape

(2, 3)

In [32]:
sides.keys()

odict_keys(['front', 'back', 'left', 'right', 'top', 'down'])

In [33]:
make_faces(n, 2)

array([[2, 0, 3],
       [1, 3, 0]])

In [34]:
# 8 edge strips
# 8 corner strips
vertical_strip = make_faces(n, 2)
horizontal_strip = make_faces(2, n)
print(vertical_strip)
print(horizontal_strip)
vertical_strip.shape, horizontal_strip.shape

[[2 0 3]
 [1 3 0]]
[[2 0 3]
 [1 3 0]]


((2, 3), (2, 3))

In [35]:
# 0 1 2 - 0 1 2
# 3 4 5 - 3 4 5
# 6 7 8 - 6 7 8
sides.keys()

odict_keys(['front', 'back', 'left', 'right', 'top', 'down'])

In [59]:
side_faces =  OrderedDict({
    'front': 0 * offset + torch.tensor(make_faces(n, n)),
    'right': 1 * offset + torch.tensor(make_faces(n, n)),
    'back' : 2 * offset + torch.tensor(make_faces(n, n)),
    'left' : 3 * offset + torch.tensor(make_faces(n, n)),
    'top'  : 4 * offset + torch.tensor(make_faces(n, n)),
    'down' : 5 * offset + torch.tensor(make_faces(n, n)),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
side_faces['front'].shape

torch.Size([2, 3])

In [60]:
front_offset = 0 * offset
#back_offse = 
corners = OrderedDict({
    'ltf': torch.tensor([(n - 1), 0, (n - 1)]),
})

In [68]:
n = 2
ids = torch.arange(start = 0, end = 6 * n**2).reshape(6, n, n)
ids = OrderedDict({    
    'front': ids[0],
    'right': ids[1],
    'back':  ids[2],
    'left':  ids[3],
    'top':   ids[4],
    'down':  ids[5],
})

#print(ids.shape)
ids

OrderedDict([('front',
              tensor([[0, 1],
                      [2, 3]])),
             ('right',
              tensor([[4, 5],
                      [6, 7]])),
             ('back',
              tensor([[ 8,  9],
                      [10, 11]])),
             ('left',
              tensor([[12, 13],
                      [14, 15]])),
             ('top',
              tensor([[16, 17],
                      [18, 19]])),
             ('down',
              tensor([[20, 21],
                      [22, 23]]))])

In [69]:
### 1 - lf0 
front, right, back, left, top, down = torch.arange(6).tolist()
offset = n ** 2

corners = OrderedDict({
    'lft': torch.tensor([
        offset * (left + 1),
        offset * (front + 1) - n,
        offset * top + 1
    ]),    
})
corners

OrderedDict([('lft', tensor([16,  2, 17]))])

In [73]:
m = trimesh.Trimesh(vertices=vert, 
    faces=torch.cat((faces, torch.tensor([[15,  2, 17]]))))
m.export('./data/cube1.stl');

In [64]:
front, right, back, left, top, down

(0, 1, 2, 3, 4, 5)

In [19]:
offset

9

In [72]:
vert

tensor([[ 1.0000, -0.5000, -0.5000],
        [ 1.0000, -0.5000,  0.5000],
        [ 1.0000,  0.5000, -0.5000],
        [ 1.0000,  0.5000,  0.5000],
        [-0.5000,  1.0000, -0.5000],
        [-0.5000,  1.0000,  0.5000],
        [ 0.5000,  1.0000, -0.5000],
        [ 0.5000,  1.0000,  0.5000],
        [-1.0000, -0.5000, -0.5000],
        [-1.0000, -0.5000,  0.5000],
        [-1.0000,  0.5000, -0.5000],
        [-1.0000,  0.5000,  0.5000],
        [-0.5000, -1.0000, -0.5000],
        [-0.5000, -1.0000,  0.5000],
        [ 0.5000, -1.0000, -0.5000],
        [ 0.5000, -1.0000,  0.5000],
        [-0.5000, -0.5000,  1.0000],
        [-0.5000,  0.5000,  1.0000],
        [ 0.5000, -0.5000,  1.0000],
        [ 0.5000,  0.5000,  1.0000],
        [-0.5000, -0.5000, -1.0000],
        [-0.5000,  0.5000, -1.0000],
        [ 0.5000, -0.5000, -1.0000],
        [ 0.5000,  0.5000, -1.0000]])

In [57]:
vert.reshape(6, 2, 2, 3)

tensor([[[[ 1.0000, -0.5000, -0.5000],
          [ 1.0000, -0.5000,  0.5000]],

         [[ 1.0000,  0.5000, -0.5000],
          [ 1.0000,  0.5000,  0.5000]]],


        [[[-1.0000, -0.5000, -0.5000],
          [-1.0000, -0.5000,  0.5000]],

         [[-1.0000,  0.5000, -0.5000],
          [-1.0000,  0.5000,  0.5000]]],


        [[[-0.5000,  1.0000, -0.5000],
          [-0.5000,  1.0000,  0.5000]],

         [[ 0.5000,  1.0000, -0.5000],
          [ 0.5000,  1.0000,  0.5000]]],


        [[[-0.5000, -1.0000, -0.5000],
          [-0.5000, -1.0000,  0.5000]],

         [[ 0.5000, -1.0000, -0.5000],
          [ 0.5000, -1.0000,  0.5000]]],


        [[[-0.5000, -0.5000,  1.0000],
          [-0.5000,  0.5000,  1.0000]],

         [[ 0.5000, -0.5000,  1.0000],
          [ 0.5000,  0.5000,  1.0000]]],


        [[[-0.5000, -0.5000, -1.0000],
          [-0.5000,  0.5000, -1.0000]],

         [[ 0.5000, -0.5000, -1.0000],
          [ 0.5000,  0.5000, -1.0000]]]])

In [50]:
6*4

24